In [3]:
# phase4_deployment_webapp.ipynb

import sys
sys.path.append(os.path.abspath("../"))
import pandas as pd
import os
from utils import visualizations

# Ensure results/logs exist
log_file = "../results/logs/chatbot_logs.csv"
session_file = "../results/reports/session_info.csv"

df = pd.read_csv(log_file, parse_dates=["timestamp"])
session_info = pd.read_csv(session_file, parse_dates=["session_start", "session_end"])


In [4]:
import ast

def parse_column(x):
    if pd.isna(x) or x == "":
        return []
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except Exception:
            return []
    return x

df["emotions"] = df["emotions"].apply(parse_column)
df["distress"] = df["distress"].apply(parse_column)


In [8]:
import ast
import numpy as np

def parse_column(x):
    # If it's a numpy array, convert to list
    if isinstance(x, np.ndarray):
        x = x.tolist()

    # Case 1: NaN
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return []

    # Case 2: Empty string
    if isinstance(x, str) and x.strip() == "":
        return []

    # Case 3: String representation of list
    if isinstance(x, str):
        try:
            parsed = ast.literal_eval(x)
            if isinstance(parsed, list):
                return parsed
            else:
                return []
        except Exception:
            return []

    # Case 4: Already a list
    if isinstance(x, list):
        return x

    # Default fallback
    return []

df["emotions"] = df["emotions"].apply(parse_column)
df["distress"] = df["distress"].apply(parse_column)


In [9]:
# Generate interactive charts
charts = visualizations.plot_combined_dashboard(session_info, df)


✅ Combined dashboard saved as combined_dashboard.html


In [10]:
# Example: Show overall distress
charts["distress"]["overall"].show()

# Example: Show daily emotion trends
charts["emotion"]["daily"].show()

# Example: Show session Gantt chart
charts["session"][1].show()


In [12]:
import ipywidgets as widgets
from IPython.display import display, clear_output

# Dropdown
distress_dropdown = widgets.Dropdown(
    options=["overall", "daily", "monthly", "yearly"],
    value="daily",
    description="Distress Trend:"
)

output = widgets.Output()

def update_distress(change):
    with output:
        clear_output(wait=True)
        charts["distress"][change.new].show()

distress_dropdown.observe(update_distress, names="value")
display(distress_dropdown, output)


Dropdown(description='Distress Trend:', index=1, options=('overall', 'daily', 'monthly', 'yearly'), value='dai…

Output()

In [ ]:
# Save individual charts to ../results/reports for web app
os.makedirs("../results/reports", exist_ok=True)

charts["distress"]["overall"].write_html("../results/reports/distress_overall.html")
charts["distress"]["daily"].write_html("../results/reports/distress_daily.html")
charts["distress"]["monthly"].write_html("../results/reports/distress_monthly.html")
charts["distress"]["yearly"].write_html("../results/reports/distress_yearly.html")

charts["emotion"]["overall"].write_html("../results/reports/emotion_overall.html")
charts["emotion"]["daily"].write_html("../results/reports/emotion_daily.html")
charts["emotion"]["monthly"].write_html("../results/reports/emotion_monthly.html")
charts["emotion"]["yearly"].write_html("../results/reports/emotion_yearly.html")

charts["combined"].write_html("../results/reports/combined_dashboard.html")
